In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import urllib.request

In [4]:
from bs4 import BeautifulSoup

In [5]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,N1 0AA,51.539442,-0.117873,Islington,Caledonian
1,N1 0EA,51.538078,-0.109623,Islington,Barnsbury
2,N1 0EU,51.537229,-0.103847,Islington,St Mary's
3,N1 0FF,51.540666,-0.126922,Camden,St Pancras and Somers Town
4,N1 0PB,51.535269,-0.103828,Islington,St Peter's


In [6]:
!pip install geocoder
import geocoder # import geocoder



In [7]:
#install geopy 
!pip install geopy
from geopy.geocoders import Nominatim
address = 'London'
geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North London are 51.5073219, -0.1276474.


In [8]:
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
!pip install folium
import folium # plotting library
import requests 

In [9]:
# create map of london using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, PostalCode in zip(df_data_4['Latitude'], df_data_4['Longitude'], df_data_4['Borough'], df_data_4['Neighborhood'], df_data_4['PostalCode']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [10]:
CLIENT_ID = 'Z54SYJPZ10HKL1DVHQJ3VSXCABSIHUGUZIRPKY3SNGB23HCK' # your Foursquare ID
CLIENT_SECRET = 'ZYW1HYBXFOHOK020EMDJKQDZXNKPQN0RR3MGVUMA4OONUFDX' # your Foursquare Secret
VERSION = '20200619' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z54SYJPZ10HKL1DVHQJ3VSXCABSIHUGUZIRPKY3SNGB23HCK
CLIENT_SECRET:ZYW1HYBXFOHOK020EMDJKQDZXNKPQN0RR3MGVUMA4OONUFDX


In [11]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# obtain London Venues near each postalcode
london_venues = getNearbyVenues(names=df_data_4['Neighborhood'],
                                   latitudes=df_data_4['Latitude'],
                                   longitudes=df_data_4['Longitude']
                                  )


Caledonian
Barnsbury
St Mary's
St Pancras and Somers Town
St Peter's
Canonbury
Mildmay
De Beauvoir
Hoxton East & Shoreditch
Hoxton West
Coppetts
Fortis Green
Alexandra
East Finchley
Muswell Hill
Southgate Green
Brunswick Park
Bounds Green
Bowes
Woodhouse
West Finchley
Totteridge
Palmers Green
Winchmore Hill
Palmers Green
Upper Edmonton
Cockfosters
Southgate
Brunswick Park
St Ann's
Tottenham Green
Seven Sisters
Stoke Newington
Clissold
Stamford Hill West
Springfield
Woodberry Down
Cazenove
Hackney Downs
Dalston
Northumberland Park
Tottenham Hale
White Hart Lane
Haselbury
Upper Edmonton
Edmonton Green
Hillrise
Tollington
Junction
St George's
St Pancras and Somers Town
St Mary's
Garden Suburb
East Finchley
Oakleigh
Totteridge
Highlands
Grange
Bush Hill Park
Woodside
Noel Park
West Finchley
Finchley Church End
St Ann's
Brownswood
Highbury West
Finsbury Park
Stroud Green
Highbury West
Highbury East
Mildmay
Crouch End
Hillrise
St George's
Holloway
Harringay
Kilburn
Hornsey
Edmonton Green
Has

In [17]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yakitori Restaurant,Yoga Studio
0,Caledonian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Caledonian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Caledonian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Caledonian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Caledonian,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# group results by neighbourhood
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yakitori Restaurant,Yoga Studio
0,Alexandra,0.0,0.00,0.000000,0.015385,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
1,Barnsbury,0.0,0.01,0.000000,0.000000,0.000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000,0.010000
2,Bounds Green,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
3,Bowes,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
4,Brownswood,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.010000,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000,0.000000
5,Brunswick Park,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
6,Bush Hill Park,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
7,Caledonian,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.020000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000,0.010000
8,Canonbury,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000,0.010000
9,Cazenove,0.0,0.00,0.000000,0.000000,0.000,0.000000,0.013514,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000,0.000000


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
# indentify top 10 categories of venue per neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexandra,Café,Pub,Coffee Shop,Pizza Place,Gym / Fitness Center,Deli / Bodega,Trail,English Restaurant,Japanese Restaurant,Restaurant
1,Barnsbury,Pub,Park,Mediterranean Restaurant,Boutique,Vietnamese Restaurant,Gastropub,Indian Restaurant,Bakery,Coffee Shop,Burger Joint
2,Bounds Green,Coffee Shop,Grocery Store,Pub,Fast Food Restaurant,Furniture / Home Store,Metro Station,Garden Center,Market,Outdoors & Recreation,Beer Bar
3,Bowes,Greek Restaurant,Grocery Store,Park,Pub,Coffee Shop,Metro Station,Food & Drink Shop,Beer Bar,Breakfast Spot,Middle Eastern Restaurant
4,Brownswood,Café,Pub,Pizza Place,Hotel,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Sports Bar,Sandwich Place,Bakery


In [21]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 2

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 1, 0, 0, 1, 1, 1], dtype=int32)

In [22]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df_data_4

# merge london_merged with neighbourhood_venues to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how = 'right')
london_merged.head() # check the last columns!


,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,N1 0AA,51.539442,-0.117873,Islington,Caledonian,1,Café,Pub,Park,Burger Joint,Gastropub,Brewery,Ethiopian Restaurant,Grocery Store,Indian Restaurant,Caucasian Restaurant
1,N1 0EA,51.538078,-0.109623,Islington,Barnsbury,1,Pub,Park,Mediterranean Restaurant,Boutique,Vietnamese Restaurant,Gastropub,Indian Restaurant,Bakery,Coffee Shop,Burger Joint
2,N1 0EU,51.537229,-0.103847,Islington,St Mary's,1,Pub,Mediterranean Restaurant,Bakery,Gastropub,Coffee Shop,Boutique,Japanese Restaurant,Park,Theater,Fish Market
51,N1P 1UP,51.539050,-0.103385,Islington,St Mary's,1,Pub,Mediterranean Restaurant,Bakery,Gastropub,Coffee Shop,Boutique,Japanese Restaurant,Park,Theater,Fish Market
3,N1 0FF,51.540666,-0.126922,Camden,St Pancras and Somers Town,1,Café,Pub,Park,Coffee Shop,Italian Restaurant,Burger Joint,Supermarket,Hotel,Grocery Store,Brewery


In [23]:
#code to plot clusters on a map of London
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [24]:
london_merged


,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,N1 0AA,51.539442,-0.117873,Islington,Caledonian,1,Café,Pub,Park,Burger Joint,Gastropub,Brewery,Ethiopian Restaurant,Grocery Store,Indian Restaurant,Caucasian Restaurant
1,N1 0EA,51.538078,-0.109623,Islington,Barnsbury,1,Pub,Park,Mediterranean Restaurant,Boutique,Vietnamese Restaurant,Gastropub,Indian Restaurant,Bakery,Coffee Shop,Burger Joint
2,N1 0EU,51.537229,-0.103847,Islington,St Mary's,1,Pub,Mediterranean Restaurant,Bakery,Gastropub,Coffee Shop,Boutique,Japanese Restaurant,Park,Theater,Fish Market
51,N1P 1UP,51.539050,-0.103385,Islington,St Mary's,1,Pub,Mediterranean Restaurant,Bakery,Gastropub,Coffee Shop,Boutique,Japanese Restaurant,Park,Theater,Fish Market
3,N1 0FF,51.540666,-0.126922,Camden,St Pancras and Somers Town,1,Café,Pub,Park,Coffee Shop,Italian Restaurant,Burger Joint,Supermarket,Hotel,Grocery Store,Brewery
50,N1C 4AA,51.536479,-0.124296,Camden,St Pancras and Somers Town,1,Café,Pub,Park,Coffee Shop,Italian Restaurant,Burger Joint,Supermarket,Hotel,Grocery Store,Brewery
4,N1 0PB,51.535269,-0.103828,Islington,St Peter's,1,Pub,Mediterranean Restaurant,Bakery,Café,Indian Restaurant,Theater,Coffee Shop,Burrito Place,French Restaurant,Park
5,N1 2DR,51.544621,-0.091108,Islington,Canonbury,1,Pub,Café,Coffee Shop,Park,Bakery,Gastropub,Italian Restaurant,Turkish Restaurant,Deli / Bodega,Theater
6,N1 2PG,51.548188,-0.092574,Islington,Mildmay,1,Pub,Café,Coffee Shop,Turkish Restaurant,Park,Restaurant,Fish & Chips Shop,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant
70,N5 2JA,51.550582,-0.089231,Islington,Mildmay,1,Pub,Café,Coffee Shop,Turkish Restaurant,Park,Restaurant,Fish & Chips Shop,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant


In [34]:
# identify neighborhoods where a pub was one of the most popular venue categories
df_pubs=pd.DataFrame(london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] +[2]+ list(range(5, london_merged.shape[1]))]])

In [37]:
df_pubs

,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51.539442,-0.117873,1,Café,Pub,Park,Burger Joint,Gastropub,Brewery,Ethiopian Restaurant,Grocery Store,Indian Restaurant,Caucasian Restaurant
1,51.538078,-0.109623,1,Pub,Park,Mediterranean Restaurant,Boutique,Vietnamese Restaurant,Gastropub,Indian Restaurant,Bakery,Coffee Shop,Burger Joint
2,51.537229,-0.103847,1,Pub,Mediterranean Restaurant,Bakery,Gastropub,Coffee Shop,Boutique,Japanese Restaurant,Park,Theater,Fish Market
51,51.539050,-0.103385,1,Pub,Mediterranean Restaurant,Bakery,Gastropub,Coffee Shop,Boutique,Japanese Restaurant,Park,Theater,Fish Market
3,51.540666,-0.126922,1,Café,Pub,Park,Coffee Shop,Italian Restaurant,Burger Joint,Supermarket,Hotel,Grocery Store,Brewery
50,51.536479,-0.124296,1,Café,Pub,Park,Coffee Shop,Italian Restaurant,Burger Joint,Supermarket,Hotel,Grocery Store,Brewery
4,51.535269,-0.103828,1,Pub,Mediterranean Restaurant,Bakery,Café,Indian Restaurant,Theater,Coffee Shop,Burrito Place,French Restaurant,Park
5,51.544621,-0.091108,1,Pub,Café,Coffee Shop,Park,Bakery,Gastropub,Italian Restaurant,Turkish Restaurant,Deli / Bodega,Theater
6,51.548188,-0.092574,1,Pub,Café,Coffee Shop,Turkish Restaurant,Park,Restaurant,Fish & Chips Shop,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant
70,51.550582,-0.089231,1,Pub,Café,Coffee Shop,Turkish Restaurant,Park,Restaurant,Fish & Chips Shop,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant


In [38]:
# find the median latitude of cluster 1 locations
df_pubs.loc[:,"Latitude"].median()

51.562795

In [39]:
# find the median longitude of cluster 1 locations
df_pubs.loc[:,"Longitude"].median()

-0.10544100000000001

In [56]:
# plot the optimal location for a distilery - at the median longitude and latitutude of cluster 1 locations
map_optimallocation = folium.Map(location=[51.562795,-0.10544100000000001], zoom_start=10)
folium.CircleMarker(
        [51.562795,-0.10544100000000001],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_optimallocation) 


In [57]:
# optimal location plotted on a map of London
map_optimallocation